<a href="https://colab.research.google.com/github/gulabpatel/Speech-to-Text/blob/main/05_Wav2Vec2_T5_VideoCaptionGenerationDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytube pydub

In [ ]:
from pytube import YouTube
from pydub import AudioSegment
import librosa
import soundfile as sf
import time
from transformers import pipeline
import requests
import pprint
pp = pprint.PrettyPrinter(indent=14)

In [ ]:
audio_path = '/content'
# link='https://www.youtube.com/watch?v=dT_aIJZ7Mxw'
link = 'https://www.youtube.com/watch?v=fr1Fz-XViIE'

try:
    # object creation using YouTube
    # which was imported in the beginning
    yt = YouTube(link)
except:
    print("Connection Error")

In [ ]:
yt.streams.filter(file_extension='mp4')

[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001f" progressive="False" type="video">, <Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="134" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.4d401e" progressive="False" type="video">, <Stream: itag="133" mime_type="video/mp4" res="240p" fps="30fps" vcodec="avc1.4d4015" progressive="False" type="video">, <Stream: itag="160" mime_type="video/mp4" res="144p" fps="30fps" vcodec="avc1.4d400c" progressive="False" type="video">, <Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">, <Stream: 

In [ ]:
stream = yt.streams.get_by_itag(139)
stream

<Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">

In [ ]:
stream.download(audio_path,"GoogleImagen.mp4")

'/content/GoogleImagen.mp4'

In [ ]:
given_audio = AudioSegment.from_file('/content/GoogleImagen.mp4', format="mp4")

In [ ]:
## convert video other foramats to wav
given_audio.export("GoogleImagen.wav", format="wav")

<_io.BufferedRandom name='GoogleImagen.wav'>

In [ ]:
sp, rate = sf.read("GoogleImagen.wav")
sp= librosa.resample(sp.T, orig_sr=rate, target_sr=16000)

In [ ]:
sf.write("GoogleImagen.wav", sp.T, 16000, subtype='PCM_24')

In [ ]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:733: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You s

In [ ]:
# Ensure that the sample rate is 16k
print(librosa.get_samplerate("GoogleImagen.wav"))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    "GoogleImagen.wav",
    block_length=5,
    frame_length=16000,
    hop_length=16000
)

16000


In [ ]:
transcribe_text = ''
for speech in stream:
        if len(speech.shape) > 1:
            speech = speech[:, 0] + speech[:, 1]

        input_values = tokenizer(speech, return_tensors="pt").input_values
        logits = model(input_values).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = tokenizer.decode(predicted_ids[0])
        transcribe_text = transcribe_text + ' ' + transcription
        print(transcription)


HALLO WELCOME TO GIOGOPE FIVE TEN A GIOBOPIC SO
AT DISIN THIS IS THE CAUSE THAT I AM TEACHING AT THE UNIVERSITY OF TENNESSEE A
ETIPAMO GEOGRAPHY AND A SUSTAINABILITY IN A SPRING TWENTY TIRTY FOUR ARIGHT NOW
AND SOT RAY IT WOULD BE THE FIRST DAY OF OUR CLASS OUR TEACHING
ARE THE THE THE THE THE CONTANT OF THIS COURSE AND I WE MAY OLLTHE VIDILS
AVAILABLE ON MYNUTOCENO SO YOU AWAKEN TO FOLLOW A AN ALSO
OR THE LEPPER SIMOND WILL ALSO BE AVAILABLE AND AM YOU CAN CHICK OUT THE CORSWEB
IE THE LING IS ALSO IN THE DESCRIPTION BELOW THOUGH THISCOURSE WE ARE
TEACHING MONEDAY WEDNESDAY FRIDAY SO IT'S A A THREE LEXUS PER WEEK
FIFTY MINUTES PUT A A PER WEEK AND YOU WACOME TO TA OTHE
CONTENT SO EXCENSORY IN THIS COURSE A WE'VE GOING TO CAVE OR HOW TO HI
VEROP OPEN SAUS A PISEMPECCIS HOW TO MAINTAIN HOW TO ECT VONCINEITY HOW TO VIGI
A DUS PETI A DU PESO DETA AND ALL WHOSARE TO QUITTE WHEPSI OR FOR THE TOCKEME
ITATION AND NO BOOK SORIKIN MIGI AH MAYE YOU WALK A ACCESSIBLE
AND ALSO REPRODUCIBLE AND ALSO

### Summarize the transcibed text

In [ ]:
## documentation for summarizer: https://huggingface.co/transformers/main_classes/pipelines.html#summarizationpipeline
# summarize with BART
# summarizer_bart = pipeline(task='summarization', model="bart-large-cnn")
# summarize with T5
summarizer_t5 = pipeline(task='summarization', model="t5-large") # options: ‘t5-small’, ‘t5-base’, ‘t5-large’, ‘t5-3b’, ‘t5-11b’
#for T5 you can chose the size of the model. Everything above t5-base is very slow, even on GPU or TPU.

In [23]:
## summarize book with BART model
t0 = time.time() # timer
summary_t5 = summarizer_t5(transcribe_text, min_length=150, max_length=1000) # change min_ and max_length for different output
print("Summarization took " + str(round((time.time() - t0) / 60, 2)) + " minutes.")

Summarization took 6.01 minutes.


In [24]:
pp.pprint(summary_t5[0]['summary_text'])

('IN THIS COURSE I AM TEACHING AT THE UNIVERSITY OF TENNESSEE A ETIPAMO '
 'GEOGRAPHY AND A SUSTAINABILITY IN A SPRING TWENTY TIRTY FOUR ARIGHT NOW AND '
 'SOT RAY IT WOULD BE THE FIRST DAY OF OUR CLASS AND WE ARE GOING TO TEACH HOW '
 "TO VIEW A PISON PECKI'S LIKE THESE A FRON GA FROM GRANAR AND I CAN ALSO  -N- "
 '-  -A- NE-A -S -E AU-N -RE-ANE-S-')


## Wav2vec2 Base model

In [27]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:733: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mod

In [28]:
# Ensure that the sample rate is 16k
print(librosa.get_samplerate("GoogleImagen.wav"))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    "GoogleImagen.wav",
    block_length=5,
    frame_length=16000,
    hop_length=16000
)

16000


In [29]:
for speech in stream:
        if len(speech.shape) > 1:
            speech = speech[:, 0] + speech[:, 1]

        input_values = tokenizer(speech, return_tensors="pt").input_values
        logits = model(input_values).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        transcription_ = tokenizer.decode(predicted_ids[0])
        transcribe_text_ = transcribe_text_ + ' ' + transcription_
        print(transcription_)

HULLO WELCOME TO JOGOPE FIVETENT A JOGPIC SOP
ER DESIGN THIS IS THE CAUSE THAT AM TEACHING AT THE UNIVERSITY OF TENNESSEE A
HE TEPAMO GEOGROPHY IN A SUSTAINABEAUTY ING A SPREIN TWENTY THIRTY FOUR ALL RIGHT NOW
AND SOTRE IT WILL BE THE FIRSTDAY OF A CLASS A TEACHING
ARE THEY THE THE THE THE CONTACT OF DISCOURT AND OW WE MAY ALL THE VEDIOS
AVILABLE ON MY EUTUC CHANNO SO YOUR WELCOME TO FOLLOW A AND ALSO
OR THE LEPER SIMOND WILL ALSO BE AVAILABLE AND TA YOU CAN CHICK OUT THE COURSE WEPT
I THE LING IS ALSO IN THE DESCRIPTION BELOW THOUGH THIS COURSE WE ARE
TEACHING MANDAY WENSDAY FRIDAY SO IS A A THREE LECTURS PER WIK
FIFTY MINUTES PU A A PE WIK AND OW YOU WAKOME TE TWOT
CONTAINCT SO EXTENTUARY IN DISCOURSE AWEARE GOING TO CAVO HAWTU
VEROPE OPEN SOUCE AR PISONPECKIES HOW TO MAINTAIN HOW TO ECONSTONALITY HOW TO VISU
AGIUSPETI E GIU SPESO DETA AND ALWO SAR TO CRI WEPSI A FOR THE DOCUME
TATION AND NOBUK SORI KI MIGI I MAKE YOU WALK I ACCESSIBLE
AND ALSO REPRODUCIBLE AND ALSO MAKE EASIER FOR O

In [32]:
## summarize book with T5 model
t0 = time.time() # timer
summary_t5_ = summarizer_t5(transcribe_text_, min_length=500, max_length=1000) # change min_ and max_length for different output
print("Summarization took " + str(round((time.time() - t0) / 60, 2)) + " minutes.")

Summarization took 13.78 minutes.


In [33]:
pp.pprint(summary_t5_[0]['summary_text'])

('JOGOPE FIVETENT A JOGPIC SOP ER DESIGN THIS IS THE CAUSE THAT I AM TEACHING '
 'AT THE UNIVERSITY OF TENNESSEE A HE TEPAMO GEOGROPHY IN A SUSTAINABEAUTY ING '
 'A SPREIN TWENTY THIRTY FOUR ALL RIGHT NOW AND SOTRE IT WILL BE THE FIRST DAY '
 'OF A CLASS A DISCOURSE LEVIE WILL BE RECORDING CLASS NE N -S -D -A AU AUX '
 'AU-N-N AUZ AN ÜZUNUNUNZ ÜÜN-UNUNN ÜNSUNUNNENEN-ZZZ AUNUNUNSO '
 'ÜNUNZUNNEUNUNSUNUNÜÜÜLUNUNLÜÜZ LUNUNSA ÜNFÜZZUNZZÜRE AN LIV AKÜÜNSTAUN AN '
 'UNANÜRE SAUXÜK AUÜÜ AN-ZÜ Ü AK-Z SKÜÜ-K AK SK AN SAU ÜS AU SÜ HER ÜKÜZ HERE '
 'HE AUAU AB AU IN AU.ZÜZÜÜ  ÜRE AKAU HE IN ÜAB ÜAU & Ü& AU& && '
 'AB-ÜÜAB-S-IN-RE-AN AN EXONAN ARE AN ZÜN ANZONAL AN DUNS AN HONS ANNON AN '
 'MÜNZ ANNAN SON ANSO AN A AN IS AN OZ ARE A ARE IN AN SO AN SA AN TAS ANNE '
 'ANSON ARE SUN ARE OAN TI ANON ARE SOME AN FAN HER-E ARE E AN WAS AB SA AU SA '
 'HER IN HER SA UND HER E HE-E-Z E AU E HERE AUE NE-NE ANN HER S-NE-E S')
